In [1]:
import requests
from bs4 import BeautifulSoup
url='https://books.toscrape.com/index.html'
response=requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')

In [2]:
ratings_dic={'One':1,'Two':2,'Three':3,'Four':4,'Five':5}

In [3]:
import csv
from functools import reduce
domain='https://books.toscrape.com/'

with open('new.csv','w',encoding='utf-8',newline='') as f:
    writer=csv.writer(f)
    writer.writerow(['Bookname','price','Rate','Category'])
    for cat in soup.find('ul',attrs={'class':''}).find_all('li'):
        cat_name=cat.find('a').get_text().strip()
        cat_second_url=cat.find('a').get('href')
        all_url=domain+cat_second_url
        cat_soup_final=BeautifulSoup(requests.get(all_url).text,'html.parser')
        
        for article in cat_soup_final.find_all('article',attrs={'class':'product_pod'}):
            name=article.find('h3').find('a').get('title')
            price=float(article.find_all('p')[1].get_text()[2:])
            rate=ratings_dic[article.find('p').get('class')[1]]
            category=cat_name
            writer.writerow([name,price,rate,category])
            
            while cat_soup_final.find('li',attrs={'class':'next'}):
                    new_page_url=reduce(lambda a,b:a+'/'+b,all_url.split('/')[:-1])+'/'+cat_soup_final.find('li',attrs={'class':'next'}).find('a').get('href')
                    cat_soup_final=BeautifulSoup(requests.get(new_page_url).text,'html.parser')
                    
                    for article in cat_soup_final.find_all('article',attrs={'class':'product_pod'}):
                        name=article.find('h3').find('a').get('title')
                        price=float(article.find_all('p')[1].get_text()[2:])
                        rate=ratings_dic[article.find('p').get('class')[1]]
                        category=cat_name
                        writer.writerow([name,price,rate,category])

In [4]:
import pandas as pd
df=pd.read_csv('new.csv')
df

Bookname  price  Rate  Category
0                              It's Only the Himalayas  45.17     2    Travel
1    Full Moon over Noahâs Ark: An Odyssey to Mou...  49.43     4    Travel
2    See America: A Celebration of Our National Par...  48.87     3    Travel
3    Vagabonding: An Uncommon Guide to the Art of L...  36.94     2    Travel
4                                 Under the Tuscan Sun  37.33     3    Travel
..                                                 ...    ...   ...       ...
995  Why the Right Went Wrong: Conservatism--From G...  52.65     4  Politics
996  Equal Is Unfair: America's Misguided Fight Aga...  56.86     1  Politics
997                                     Amid the Chaos  36.58     1  Cultural
998                                         Dark Notes  19.19     5   Erotica
999  The Long Shadow of Small Ghosts: Murder and Me...  10.97     1     Crime

[1000 rows x 4 columns]